In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import zarr
import gcsfs

In [ ]:
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
# df_ta = df.query("activity_id=='CMIP' & table_id == 'Amon' & variable_id == 'tas' & experiment_id == 'historical'")

In [ ]:
df

[link](https://nbviewer.org/github/pangeo-data/pangeo-cmip6-examples/blob/master/basic_search_and_load.ipynb?source=post_page-----96b177abe396---------------------------------------)